In [1]:
import os

import itertools
import pickle
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math 

import sys

%matplotlib inline

In [2]:
import torchtext

In [3]:
CHAR_FIELD = torchtext.data.Field(lower=True, tokenize=lambda w: list(w))
train_set, val_set, test_set = torchtext.datasets.WikiText2.splits(CHAR_FIELD)
CHAR_FIELD.build_vocab(train_set)
train_iter, val_iter, test_iter = torchtext.data.BPTTIterator.splits((train_set, val_set, test_set), batch_size=128, bptt_len=30)

In [4]:
VOCAB_LEN = len(CHAR_FIELD.vocab.itos)

In [5]:
class RNNModel(nn.Module):

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, hidden=None):
        emb = self.drop(self.encoder(x))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (weight.new(self.nlayers, bsz, self.nhid).zero_(),
                    weight.new(self.nlayers, bsz, self.nhid).zero_())
        else:
            return weight.new(self.nlayers, bsz, self.nhid).zero_()

In [6]:
def train(model, train_iter, ntokens=VOCAB_LEN):
    model.train()
    total_loss = 0
    
    for batch, data in enumerate(train_iter):
        model.zero_grad()
        output, hidden = model(data.text)
        loss = criterion(output.view(-1, ntokens), data.target.view(-1))
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_iter), lr, cur_loss, math.exp(cur_loss)))
            total_loss = 0

In [7]:
def evaluate(model, data_iter, ntokens=VOCAB_LEN):
    model.eval()
    total_loss = 0
    
    hidden = model.init_hidden(eval_batch_size)
    for i, data in enumerate(data_iter):
        output, hidden = model(data.text)
        output_flat = output.view(-1, ntokens)
        total_loss += len(data) * criterion(output_flat, data.target.view(-1)).item()
    return total_loss / len(data_iter)

In [8]:
def generate(model, itos, n=50, temp=1., ntokens=VOCAB_LEN):
    model.eval()
    x = torch.rand(1, 1).mul(ntokens).long()
    hidden = None
    out = []
    for i in range(n):
        output, hidden = model(x, hidden)
        s_weights = output.squeeze().data.div(temp).exp()
        s_idx = torch.multinomial(s_weights, 1)[0]
        x.data.fill_(s_idx)
        s = itos[s_idx]
        out.append(s)
    return ''.join(out)

In [9]:
model = RNNModel('LSTM', VOCAB_LEN, 128, 128, 2, 0.3)
criterion = nn.CrossEntropyLoss()

In [10]:
batch_size = 128
sequence_length = 30
grad_clip = 0.1
lr = 4.
best_val_loss = None
log_interval = 100
eval_batch_size = 128

In [11]:
with torch.no_grad():
    print('sample:\n', generate(model, CHAR_FIELD.vocab.itos, 50), '\n')

for epoch in range(1, 6):
    train(model, train_iter)
    val_loss = evaluate(model, val_iter)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(model, CHAR_FIELD.vocab.itos, 50), '\n')

sample:
 ยルμ−„礮⅓ệṯư°wュ-プpვệpż₤€ăäh大.o·ōス&þ̃هキ♭̃าê・ッ〉v>3v่รu 

| epoch   1 |   100/ 2808 batches | lr 4.00 | loss  3.46 | ppl    31.75
| epoch   1 |   200/ 2808 batches | lr 4.00 | loss  3.15 | ppl    23.40
| epoch   1 |   300/ 2808 batches | lr 4.00 | loss  3.12 | ppl    22.76
| epoch   1 |   400/ 2808 batches | lr 4.00 | loss  3.10 | ppl    22.26
| epoch   1 |   500/ 2808 batches | lr 4.00 | loss  3.10 | ppl    22.30
| epoch   1 |   600/ 2808 batches | lr 4.00 | loss  3.09 | ppl    22.08
| epoch   1 |   700/ 2808 batches | lr 4.00 | loss  3.06 | ppl    21.29
| epoch   1 |   800/ 2808 batches | lr 4.00 | loss  2.90 | ppl    18.20
| epoch   1 |   900/ 2808 batches | lr 4.00 | loss  2.77 | ppl    15.90
| epoch   1 |  1000/ 2808 batches | lr 4.00 | loss  2.65 | ppl    14.14
| epoch   1 |  1100/ 2808 batches | lr 4.00 | loss  2.54 | ppl    12.71
| epoch   1 |  1200/ 2808 batches | lr 4.00 | loss  2.47 | ppl    11.86
| epoch   1 |  1300/ 2808 batches | lr 4.00 | loss  2.42 | ppl    11.23
| 

| epoch   4 |  1300/ 2808 batches | lr 4.00 | loss  1.71 | ppl     5.50
| epoch   4 |  1400/ 2808 batches | lr 4.00 | loss  1.69 | ppl     5.41
| epoch   4 |  1500/ 2808 batches | lr 4.00 | loss  1.70 | ppl     5.46
| epoch   4 |  1600/ 2808 batches | lr 4.00 | loss  1.70 | ppl     5.48
| epoch   4 |  1700/ 2808 batches | lr 4.00 | loss  1.70 | ppl     5.45
| epoch   4 |  1800/ 2808 batches | lr 4.00 | loss  1.70 | ppl     5.46
| epoch   4 |  1900/ 2808 batches | lr 4.00 | loss  1.71 | ppl     5.52
| epoch   4 |  2000/ 2808 batches | lr 4.00 | loss  1.69 | ppl     5.44
| epoch   4 |  2100/ 2808 batches | lr 4.00 | loss  1.70 | ppl     5.49
| epoch   4 |  2200/ 2808 batches | lr 4.00 | loss  1.70 | ppl     5.45
| epoch   4 |  2300/ 2808 batches | lr 4.00 | loss  1.70 | ppl     5.48
| epoch   4 |  2400/ 2808 batches | lr 4.00 | loss  1.69 | ppl     5.40
| epoch   4 |  2500/ 2808 batches | lr 4.00 | loss  1.69 | ppl     5.41
| epoch   4 |  2600/ 2808 batches | lr 4.00 | loss  1.70 | ppl  

In [16]:
print("Val loss = {:.2f}, val ppl = {:.2f}".format(val_loss / 128, math.exp(val_loss / 128)))

Val loss = 1.45, val ppl = 4.27
